In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision import transforms
import cv2
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
# from sklearn.cross_validation import cross_val_score

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/ranger/ranger')
import timm
from ranger import Ranger
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:

# !git clone https://github.com/shengliu66/ELR ../input/ELR_Loss
    
# sys.path.append('./ELR_Loss')


In [ ]:
# !pip install -r requirements.txt
# from ELR.model.loss import elr_loss

In [ ]:
cfg = {}
cfg['epoch'] = 20
cfg['batch_size'] = 16
cfg['lr'] = 0.016
cfg['image_size'] = 512 # (600, 800)
cfg['device'] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg['fold'] = 1
cfg['split'] = 0.8
cfg['smoothing_factor'] = 0.01
TRAINDATAPATH = '../input/cassava-leaf-disease-classification/train_images'
TESTDATAPATH = '../input/cassava-leaf-disease-classification/test_images'
TRAINLABELPATH = '../input/cassava-leaf-disease-classification/train.csv'
TESTSAMPLEPATH = '../input/cassava-leaf-disease-classification/sample_submission.csv'
CLASSESJSONPATH = '../input/cassava-leaf-disease-classification/label_num_to_disease_map.json'

classes = {}
num_class = 0

In [ ]:
seed=31

def setSeed(seed=31):
#     random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    pd.core.common.random_state(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

setSeed(31)

## get the class

In [ ]:
import json

with open(CLASSESJSONPATH) as f:
    global classses, num_class
    classes = json.load(f)
    num_class = len(classes)

## utils

In [ ]:
# read dataset
train_df = pd.read_csv(TRAINLABELPATH)


def readImage(ID, path):
    ID = ID.split('.')[0]
    filepath = os.path.join(path, ID + '.jpg')
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


In [ ]:
# img = readImage('../input/cassava-leaf-disease-classification/train_images', '1000015157.jpg')


## show data/ pictures

In [ ]:
# show data
from collections import Counter
def showData(df):
    display(df.info())
    
    c = Counter(df.label)
    display(c)
    
    plt.clf()
    plt.pie(c.values(), labels=c.keys(),autopct='%1.2f%%')
    plt.show()
    
# showData(train_df)
def showExample(df):
    num_show = 4
    plt.clf()
    f, axis = plt.subplots(num_class, num_show, figsize=(6 * num_show, 25))
    for cls, name in classes.items():
        idx = int(cls)
#         print(cls)
        sampleimgs = df[df['label'] == idx]['image_id']
        rndidxs = np.random.randint(len(sampleimgs), size=num_show)
        for i, randnum in enumerate(rndidxs):
            axis[idx,i].set_title(name)
            axis[idx,i].imshow(readImage(sampleimgs.iloc[i], TRAINDATAPATH))
        
    plt.show()


In [ ]:
# showExample(train_df)

## dataset, dataloader

In [ ]:
# dataset and data loader
class CLCdataset(Dataset):
    def __init__(self, df,isTrain = True):
        self.df = df
        self.isTrain = isTrain
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        ID = self.df['image_id'].iloc[idx].split('.')[0]
        img = readImage(ID, TRAINDATAPATH if self.isTrain else TESTDATAPATH)
        if self.isTrain:
            img = train_transform(image = img)['image']
        else:
            img = test_transform(image = img)['image']
        if self.isTrain:
            label = self.df['label'].iloc[idx]
            return ID, img, label
        return ID, img

In [ ]:
# transform

train_transform = A.Compose([
    A.Flip(p=0.5),
    A.Resize(cfg['image_size'], cfg['image_size'],p=1),
    A.ShiftScaleRotate(p=0.5),
    A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2() # (H,W,C) to (C,H,W)
])

test_transform = A.Compose([
    A.Resize(cfg['image_size'], cfg['image_size'],p=1),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2() # (H,W,C) to (C,H,W)
])


## model

In [ ]:
class CFCmodel(nn.Module):
    def __init__(self, out_dim):
        super(CFCmodel, self).__init__()
        self.model = timm.create_model('efficientnet_b3', pretrained=True)
        self.model.classifier = nn.Linear(in_features=1536,out_features=out_dim, bias=True)
        self.model.eval()
#         self.classifier = nn.Sequential(nn.Linear(1000, out_dim),
#                                         nn.Softmax())
#         self.classifier = nn.Sequential(nn.Linear(1000, out_dim))
        # freeze attibute
#         for p in self.features.parameters():
#             p.requires_grad = False
    def forward(self, input):
        return self.model(input)

In [ ]:
# import numpy as np
# models = timm.create_model('efficientnet_b3', pretrained=False)
# models.classifier = nn.Linear(in_features=1536,out_features=5, bias=True)
# print(models)


In [ ]:
# label smoothing loss
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.01, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes
        self.dim = dim 
    def forward(self, pred, target): 
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad(): 
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
    
class LabelSmoothingCrossEntropy(nn.Module):

    def __init__(self, epsilon: float = 0.1):
        super().__init__()
        self.epsilon = epsilon

    def linear_combination(self, x, y, epsilon):
        return epsilon*x + (1-epsilon)*y

    def reduce_loss(self, loss, reduction='mean'):
        return loss.mean() if reduction == 'mean' else loss.sum() if reduction == 'sum' else loss

    def forward(self, preds, target, reduction='mean'):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = self.reduce_loss(-log_preds.sum(dim=-1), reduction=reduction)
        nll = F.nll_loss(log_preds, target, reduction=reduction)
        return self.linear_combination(loss/n, nll, self.epsilon)

In [ ]:
def get_loss(name):
    loss_dict = {
        'CrossEntropy': F.cross_entropy,
        'LabelSmoothingCrossEntropy':
        LabelSmoothingCrossEntropy(epsilon=0.1)
    }
    return loss_dict[name]

class OUSMLoss(nn.Module):
    '''
    Implementation of 
    Loss with Online Uncertainty Sample Mining:
    https://arxiv.org/pdf/1901.07759.pdf
    # Params
    k: num of samples to drop in a mini batch
    loss: loss function name (see get_loss function above)
    trigger: the epoch it starts to train on OUSM (please call `.update(epoch)` each epoch)
    '''

    def __init__(self, k=1, loss='LabelSmoothingCrossEntropy', trigger=2, ousm=False):
        super(OUSMLoss, self).__init__()
        self.k = k
        self.loss_name = loss
        self.loss = get_loss(loss)
        self.trigger = trigger
        self.ousm = ousm

    def forward(self, logits, targets, indices=None):
        bs = logits.shape[0]
        if self.ousm and bs - self.k > 0:
            losses = self.loss(logits, targets, reduction='none')
            if len(losses.shape) == 2:
                losses = losses.mean(1)
            _, idxs = losses.topk(bs-self.k, largest=False)
            losses = losses.index_select(0, idxs)
            return losses.mean()
        else:
            return self.loss(logits, targets)

    def update(self, current_epoch):
        self.current_epoch = current_epoch
        if current_epoch == self.trigger:
            self.ousm = True
            print('criterion: ousm is True.')

    def __repr__(self):
        return f'OUSM(loss={self.loss_name}, k={self.k}, trigger={self.trigger}, ousm={self.ousm})'

In [ ]:
def accuracy_score(pred, label):
    assert len(pred) == len(label), 'Shape not match! pred:{}, label:{}, batch size: {}'.format(np.shape(pred),np.shape(label),cfg['batch_size'])
    lengh = len(pred)
    tt_acc = 0
    for (i, j) in zip(pred, label):
        tt_acc += 1 if i == j else 0
    return tt_acc / lengh

In [ ]:
## train / valid a epoch

In [ ]:
def train(model, dataloader, criterion, optimizer, scheduler):
    model.train()
    totalloss = 0
    totalacc = 0
    with tqdm(dataloader,unit='batch',desc='Train') as tqdm_loader:
        for idx, (ID, img, label) in enumerate(tqdm_loader):
            img = img.to(device=cfg['device'])
            label = label.to(device=cfg['device'])
            label = torch.tensor(label, dtype=torch.long) 
#             print(f'img:\n{img}')
            pred = model(img).to(device=cfg['device'])
#             print(f'pred:\n{pred} label:\n{label}')
            loss = criterion(pred, label)
            pred = pred.cpu().detach().argmax(dim=1)

            optimizer.zero_grad()
            loss.backward()           
            optimizer.step()
#             scheduler.step()
            
            nowloss = loss.detach().item()
            totalloss += nowloss
            
            acc = accuracy_score(pred, label.cpu())
            totalacc += acc
            
            tqdm_loader.set_postfix(loss=nowloss,avgloss=totalloss/(idx+1),avgACC=totalacc/(idx+1) )

def valid(model, dataloader, certification, fold, epoch):
    model.eval()
    totalloss=0
    totalacc=0
    with torch.no_grad():
        with tqdm(dataloader,unit='batch',desc='Valid') as tqdm_loader:
            for idx, (ID, img, label) in enumerate(tqdm_loader):
                img = img.to(device=cfg['device'])
#                 label = label.to(device=cfg['device'])
                label = torch.tensor(label, dtype=torch.long).to(device=cfg['device'])
                
                pred = model(img).to(device=cfg['device'])
                
                loss = certification(pred, label)
                
                pred = pred.cpu().detach().argmax(dim=1)
                
                nowloss = loss.detach().item()
                totalloss += nowloss
                
                acc = accuracy_score(pred, label.cpu())
                totalacc += acc

                tqdm_loader.set_postfix(loss=nowloss,avgloss=totalloss/(idx+1),avgACC=totalacc/(idx+1) )
            bestacc = totalacc/len(tqdm_loader)
            bestloss = totalloss/len(tqdm_loader)
    return bestacc, bestloss

In [ ]:

def plotLosses(losses):
    plt.clf()
    plt.title("Losses")
    plt.plot(losses)
    plt.savefig("lossfig.jpg")

## Cross validation train

In [ ]:
def cv_train(dataset):
    frac = int(len(dataset) / cfg['fold'])
    accs = []
    for fold in range(cfg['fold']):
        print(f'\nFold {fold}')
        if cfg['fold'] != 1:
            train_rg = list(range(0, fold * frac)) + list(range((fold+1) * frac, len(dataset)) )  
            valid_rg = list(range(fold * frac, (fold+1) * frac))
        else:
            train_rg = list(range(0, int(cfg['split'] * len(dataset))))
            valid_rg = list(range(int(cfg['split'] * len(dataset)), len(dataset)))
#         print(train_rg)
#         print(valid_rg)
        
        train_dataset = torch.utils.data.Subset(dataset, train_rg)
        train_dataset.isTrain = True
        valid_dataset = torch.utils.data.Subset(dataset, valid_rg)
        valid_dataset.isTrain = False
#         print(train_dataset)
#         print(valid_dataset)
        train_dataloader = DataLoader(train_dataset, batch_size=cfg['batch_size'], shuffle=True, num_workers=2)
        valid_dataloader = DataLoader(valid_dataset, batch_size=cfg['batch_size'], shuffle=False, num_workers=2)
        
        model = CFCmodel(num_class).to(cfg['device'])

#         criterion = LabelSmoothingLoss(classes=num_class, smoothing=cfg['smoothing_factor'])
        criterion = OUSMLoss()
        optimizer = Ranger(model.parameters())
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max= 10, eta_min=1e-6)
#         optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
#         scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr=0.1, steps_per_epoch=len(train_dataloader), epochs=cfg['epoch'])
        best_fold_model = -1
        best_fold_loss = 100
        losses = []
        for epoch in range(cfg['epoch']):
            print(f'\nEpoch {epoch}')
            train(model, train_dataloader, criterion, optimizer, scheduler)
            acc, loss = valid(model, valid_dataloader, criterion, fold, epoch)
            if loss <= best_fold_loss:
                best_fold_model = epoch
                best_fold_loss = loss
            torch.save(model.state_dict(), os.path.join('./', f'model{fold}_{epoch}.pth'))
                
            accs.append(acc)
            losses.append(loss)
            scheduler.step()
            criterion.update(epoch)
        print(f'fold {fold} best epoch: {best_fold_model}')
        plotLosses(losses)
#     print(accs)
    print(f'ac_score: {np.mean(accs)}')
    

In [ ]:
# torch.save(model.state_dict(), os.path.join('./', f'best_model0_9.pth'))

In [ ]:
train_dataset = CLCdataset(train_df)
cv_train(train_dataset)